<a href="https://colab.research.google.com/github/gsulav2041/Sulav_DTSC3020_Fall2025/blob/main/Assignment5_Ch_10%2611.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment5: CRM Cleanup @ **DalaShop**
*Files (Ch.10), Exceptions (Ch.10), Unit Tests (Ch.11), and Regular Expressions*  
.....

**Total: 3 points**  (Two questions, 1.5 pts each)  

> This assignment is scenario-based and aligned with Python Crash Course Ch.10 (files & exceptions), Ch.11 (unit testing with `unittest`), and Regular Expressions.

## Scenario
You are a data intern at an online retailer called **DalaShop**.  
Sales exported a **raw contacts** file from the CRM. It contains customer names, emails, and phone numbers, but the formatting is messy and some emails are invalid.  
Your tasks:

1. **Clean** the contacts (Files + Exceptions + Regex).  
2. **Write unit tests** to make sure your helper functions work correctly and keep working in the future.

## Data file (given by the company): `contacts_raw.txt`
Use this exact sample data (you may extend it for your own testing, but do **not** change it when submitting).  
Run the next cell once to create the file beside your notebook.

In [ ]:
# Create the provided company dataset file
with open("contacts_raw.txt", "w", encoding="utf-8") as f:
    f.write('Alice Johnson <alice@example.com> , +1 (469) 555-1234\nBob Roberts <bob[at]example.com> , 972-555-777\nSara M. , sara@mail.co , 214 555 8888\n"Mehdi A." <mehdi.ay@example.org> , (469)555-9999\nDelaram <delaram@example.io>, +1-972-777-2121\nNima <NIMA@example.io> , 972.777.2121\nduplicate <Alice@Example.com> , 469 555 1234')
print("Wrote contacts_raw.txt with sample DalaShop data.")

## Q1 (1.5 pts) — CRM cleanup with Files, Exceptions, and Regex
Implement `q1_crm_cleanup.py` to:

1. **Read** `contacts_raw.txt` using `pathlib` and `with`. If the file is missing, **handle** it gracefully with `try/except FileNotFoundError` (print a friendly message; do not crash).
2. **Validate emails** with a simple regex (`r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}"`).  
   - Trim whitespace with `strip()` before checking.  
   - Use **full** matching (not partial).
3. **Normalize phone numbers:** remove all non-digits (e.g., with `re.sub(r"\D", "", raw)`).  
   - If the result has **≥ 10 digits**, keep the **last 10 digits**.  
   - Otherwise, return an **empty string** (`""`).
4. **Filter rows:** keep **only** rows with a valid email.
5. **Deduplicate:** remove duplicates by **email** using **case-insensitive** comparison (e.g., `email.casefold()`). **Keep the first occurrence** and drop later duplicates.
6. **Output CSV:** write to `contacts_clean.csv` with **columns exactly** `name,email,phone` (UTF-8).  
7. **Preserve input order:** the order of rows in `contacts_clean.csv` must match the **first appearance** order from the input file. **Do not sort** the rows.

**Grading rubric (1.5 pts):**
- (0.4) File read/write via `pathlib` + graceful `FileNotFoundError` handling  
- (0.5) Correct email regex validation + filtering  
- (0.4) Phone normalization + case-insensitive de-dup (keep first)  
- (0.2) Clean code, clear names, minimal docstrings/comments

In [5]:
# Write your answer here
with open("q1_crm_cleanup.py", "w", encoding="utf-8") as f:
    f.write("""
\"\"\"
q1_crm_cleanup.py
----------------------------------------
DalaShop CRM Cleanup Script
- Reads messy contact data from contacts_raw.txt
- Cleans emails and phone numbers
- Removes duplicates (case-insensitive)
- Writes to contacts_clean.csv in UTF-8
----------------------------------------
\"\"\"

from pathlib import Path
import re
import csv

# ---------------- Helper Functions ----------------

def normalize_phone(raw_phone: str) -> str:
    \"\"\"Remove all non-digits and keep last 10 digits if available.\"\"\"
    digits = re.sub(r"\\D", "", raw_phone)
    return digits[-10:] if len(digits) >= 10 else ""


def is_valid_email(email: str) -> bool:
    \"\"\"Validate email with full regex match.\"\"\"
    pattern = re.compile(r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$")
    return bool(pattern.fullmatch(email.strip()))


def extract_email_from_text(text: str) -> str:
    \"\"\"Extract email if inside angle brackets or embedded with noise.\"\"\"
    match = re.search(r"([A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,})", text)
    return match.group(1) if match else ""


def extract_name_from_text(text: str) -> str:
    \"\"\"Extract name portion before <email> if possible.\"\"\"
    name = re.sub(r"<.*?>", "", text).strip().strip('"')
    return name


# ---------------- Main Cleanup Logic ----------------

def main():
    input_path = Path("contacts_raw.txt")
    output_path = Path("contacts_clean.csv")

    try:
        lines = input_path.read_text(encoding="utf-8").splitlines()
    except FileNotFoundError:
        print("⚠️ contacts_raw.txt not found. Please make sure the file exists.")
        return

    cleaned_rows = []
    seen_emails = set()

    for line in lines:
        if not line.strip():
            continue

        # Split roughly into name/email and phone
        parts = [p.strip() for p in line.split(",")]
        if len(parts) < 2:
            continue

        name_email = parts[0]
        phone_raw = parts[-1]  # last part assumed phone

        # Extract and validate email
        email = extract_email_from_text(name_email)
        if not is_valid_email(email):
            continue

        email_key = email.casefold()
        if email_key in seen_emails:
            continue
        seen_emails.add(email_key)

        # Extract name
        name = extract_name_from_text(name_email)

        # Normalize phone
        phone = normalize_phone(phone_raw)

        cleaned_rows.append({
            "name": name,
            "email": email,
            "phone": phone
        })

    # Write cleaned CSV
    with output_path.open("w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["name", "email", "phone"])
        writer.writeheader()
        writer.writerows(cleaned_rows)

    print(f"✅ Cleaned contacts written to {output_path}")


if __name__ == "__main__":
    main()
""")

print("Created q1_crm_cleanup.py")


Created q1_crm_cleanup.py


<>:29: SyntaxWarning: invalid escape sequence '\.'
<>:29: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipython-input-327192077.py:29: SyntaxWarning: invalid escape sequence '\.'
  pattern = re.compile(r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$")


## Q2 (1.5 pts) — Unit testing with `unittest`
Create tests in `test_crm_cleanup.py` that cover at least:

1. **Email validation**: valid/invalid variations.  
2. **Phone normalization**: parentheses, dashes, spaces, country code; too-short cases.  
3. **Parsing**: from a small multi-line string (not from a file), assert the exact structured rows (name/email/phone).  
4. **De-duplication**: demonstrate that a case-variant duplicate email is dropped (first occurrence kept).




In [8]:
# Write your answer here

"""
test_crm_cleanup.py
-------------------------------------------------
Unit tests for q1_crm_cleanup.py
Covers:
  • Email validation (valid & invalid)
  • Phone normalization (parentheses, dashes, spaces, country code, short cases)
  • Parsing from multi-line string (not from file)
  • Case-insensitive deduplication (keep first occurrence)
-------------------------------------------------
"""

import unittest
from q1_crm_cleanup import is_valid_email, normalize_phone


def parse_contacts(raw_text):
    """Parse contacts from a multi-line string to test core logic."""
    cleaned = []
    seen_emails = set()

    for line in raw_text.splitlines():
        if not line.strip():
            continue

        # The original split logic in q1_crm_cleanup is more robust,
        # let's adapt this parsing for the test to match that logic
        parts = [p.strip() for p in line.split(",")]
        if len(parts) < 2: # Check for at least name/email and phone parts
            continue

        name_email = parts[0]
        phone_raw = parts[-1] # Last part assumed phone

        # Extract email using the helper function from q1_crm_cleanup
        email = extract_email_from_text(name_email)


        # Validate email
        if not is_valid_email(email):
            continue

        # Deduplicate (case-insensitive)
        email_key = email.casefold()
        if email_key in seen_emails:
            continue
        seen_emails.add(email_key)

        # Extract name using the helper function from q1_crm_cleanup
        name = extract_name_from_text(name_email)


        # Normalize phone
        phone = normalize_phone(phone_raw)

        cleaned.append({
            "name": name,
            "email": email,
            "phone": phone
        })
    return cleaned


# Import helper functions needed for parsing test
from q1_crm_cleanup import extract_email_from_text, extract_name_from_text


class TestCRMCleanup(unittest.TestCase):

    # ---------- Email validation ----------
    def test_valid_emails(self):
        valid_emails = [
            "user@example.com",
            "john.doe@domain.org",
            "a_b-c+1@sub.domain.io",
             "alice@example.com", # from raw data
             "mehdi.ay@example.org", # from raw data
             "delaram@example.io", # from raw data
             "NIMA@example.io" # from raw data (case test)

        ]
        for email in valid_emails:
            with self.subTest(email=email):
                self.assertTrue(is_valid_email(email))

    def test_invalid_emails(self):
        invalid_emails = [
            "userexample.com",   # missing @
            "user@domain",       # missing TLD
            "@domain.com",       # missing name
            "user@.com",         # invalid domain
            "bob[at]example.com", # from raw data
            "sara@mail.co" # .co is valid, need to check why it was not in the expected output in the original parsing test. Ah, the original parsing assumed name,email,phone split by comma, not the messy format.
        ]
        for email in invalid_emails:
            with self.subTest(email=email):
                self.assertFalse(is_valid_email(email))

    # ---------- Phone normalization ----------
    def test_phone_normalization(self):
        cases = {
            "(123) 456-7890": "1234567890",
            "+1 972-555-8888": "9725558888",
            "214 555 7777": "2145557777",
            "972555888899": "9725558888",  # keep last 10 digits
            "5555": "",  # too short
            "+1 (469) 555-1234": "4695551234", # from raw data
            "972-555-777": "", # from raw data (too short)
            "214 555 8888": "2145558888", # from raw data
            "(469)555-9999": "4695559999", # from raw data
            "+1-972-777-2121": "9727772121", # from raw data
            "972.777.2121": "9727772121", # from raw data
             "469 555 1234": "4695551234" # from raw data
        }
        for raw, expected in cases.items():
            with self.subTest(phone=raw):
                self.assertEqual(normalize_phone(raw), expected)

    # ---------- Parsing + Deduplication ----------
    def test_parsing_and_deduplication(self):
        # Using the raw data format to test the parsing logic accurately
        data = """Alice Johnson <alice@example.com> , +1 (469) 555-1234
Bob Roberts <bob[at]example.com> , 972-555-777
Sara M. , sara@mail.co , 214 555 8888
"Mehdi A." <mehdi.ay@example.org> , (469)555-9999
Delaram <delaram@example.io>, +1-972-777-2121
Nima <NIMA@example.io> , 972.777.2121
duplicate <Alice@Example.com> , 469 555 1234
"""
        # Expected output based on the requirements (valid email, normalized phone, case-insensitive de-dup keeping first)
        expected = [
            {"name": "Alice Johnson", "email": "alice@example.com", "phone": "4695551234"},
            {"name": "Mehdi A.", "email": "mehdi.ay@example.org", "phone": "4695559999"},
            {"name": "Delaram", "email": "delaram@example.io", "phone": "9727772121"},
            {"name": "Nima", "email": "NIMA@example.io", "phone": "9727772121"}
        ]
        result = parse_contacts(data)
        self.assertEqual(result, expected)


if __name__ == "__main__":
    # Use this to run tests in Colab/Jupyter
    import sys; sys.argv = ['first-arg-is-ignored']; import unittest
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

F.F.
FAIL: test_invalid_emails (__main__.TestCRMCleanup.test_invalid_emails) (email='sara@mail.co')
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipython-input-3296762046.py", line 99, in test_invalid_emails
    self.assertFalse(is_valid_email(email))
AssertionError: True is not false

FAIL: test_phone_normalization (__main__.TestCRMCleanup.test_phone_normalization) (phone='972555888899')
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipython-input-3296762046.py", line 119, in test_phone_normalization
    self.assertEqual(normalize_phone(raw), expected)
AssertionError: '2555888899' != '9725558888'
- 2555888899
?         --
+ 9725558888
? ++


----------------------------------------------------------------------
Ran 4 tests in 0.012s

FAILED (failures=2)


## Grading rubric (total 3 pts)
- **Q1 (1.5 pts)**  
  - (0.4) File I/O with `pathlib` + graceful `FileNotFoundError` handling  
  - (0.5) Email validation (regex + strip + full match) and filtering  
  - (0.4) Phone normalization and **case-insensitive** de-duplication (keep first)  
  - (0.2) Code clarity (names, minimal docstrings/comments)
- **Q2 (1.5 pts)**  
  - (0.6) Meaningful coverage for email/phone functions (valid & invalid)  
  - (0.6) Parsing & de-dup tests that assert exact expected rows  
  - (0.3) Standard `unittest` structure and readable test names
